# Section plots

So pretty and useful!

In [ ]:
import emsarray
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from emsarray.plot import plot_on_figure, bounds_to_extent
from matplotlib import cm
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.patheffects import withStroke
from shapely.geometry import LineString, Point

In [ ]:
%matplotlib inline

# This makes the figures nice and big for this notebook
from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['savefig.dpi'] = 100

# The coastlines used in the maps have some bad polygons,
# which causes some warnings. Lets turn those off.
import warnings
warnings.filterwarnings(
    "ignore", message="Shapefile shape has invalid polygon",
    category=UserWarning, module="shapefile")

In [ ]:
line = LineString([
    [147.2888, -42.7910],
    [147.2713, -42.8003],
    [147.3088, -42.8220],
    [147.3248, -42.8311],
    [147.3299, -42.8399],
    [147.3375, -42.8565],
    [147.3497, -42.8821],
    [147.3820, -42.9015],
    [147.3832, -42.9440],
    [147.3605, -43.0011],
    [147.3874, -43.0637],
    [147.4908, -43.1546],
])

landmarks = [
    ("Bowen Bridge", Point(147.306646, -42.818007)),
    # ("Tasman Bridge", Point(147.343568, -42.864934)),
    ("Battery Point", Point(147.339205, -42.889375)),
    ("Kingston", Point(147.326664, -42.979781)),
    ("Iron Pot", Point(147.416533, -43.058994)),
]

dataset = emsarray.tutorial.open_dataset("ugrid_mesh2d")
# dataset['Mesh2_layers'].attrs['positive'] = 'up'
dataset

In [ ]:
figure = plt.figure(figsize=(10, 6), dpi=100)
plot_on_figure(figure, dataset.ems, scalar=dataset["Mesh2_depth"], title="Bathymetry")

axes = figure.axes[0]
axes.plot(*line.xy, color="blue")
axes.set_extent(bounds_to_extent(line.buffer(0.1).bounds))

outline = withStroke(linewidth=2, foreground='w')
axes.plot(
    [p.x for l, p in landmarks], [p.y for l, p in landmarks], 'o',
    markersize=3, color='black', path_effects=[outline])
for label, point in landmarks:
    axes.annotate(
        label, (point.x, point.y),
        xytext=(5, 0), textcoords='offset points',
        color='black', path_effects=[outline], verticalalignment='center_baseline')

In [ ]:
figure = plt.figure(figsize=(12, 3), dpi=100)
section = sections.SectionPlot(dataset.ems, line, depth=dataset["Mesh2_layers"])
section.plot_on_figure(
    figure=figure,
    data_array=dataset["temp"].isel(record=0),
    bathymetry=dataset["Mesh2_depth"],
    landmarks=landmarks,
)
figure.tight_layout()

In [ ]:
from emsarray import sections
from emsarray.utils import datetime_from_np_time

from IPython.display import HTML
import os.path
from matplotlib import animation

def make_title(np_time):
    """Generate a title for a frame of animation."""
    long_name = "Temperature"
    py_time = datetime_from_np_time(np_time)
    return f'{long_name}\nTime: {py_time:%Y-%m-%d %H:%M}'

dataset = emsarray.open_dataset("/home/hea211/example-datasets/setas_compas_all_2018-05.nc")
ds = dataset

figure = plt.figure(figsize=(12, 3), dpi=100)
figure.patch.set_facecolor('white')
figure.patch.set_alpha(1)
figure.transparent = False

section = sections.SectionPlot(format=ds.ems, line=line, depth=ds["Mesh2_layers"])
anim = section.animate_on_figure(
    figure=figure,
    coordinate=ds["t"],
    data_array=ds["temp"],
    bathymetry=dataset["Mesh2_depth"],
    title=make_title,
    landmarks=landmarks,
)
figure.tight_layout()

writer = animation.FFMpegFileWriter(bitrate=-1, codec='mpeg4')
video_path = os.path.abspath("../section-animation.mp4")
anim.save(video_path, writer=writer)
print("Saved to", video_path)

gif_path = os.path.abspath("../section-animation.gif")
anim.save(gif_path)
print("Saved to", gif_path)